<a href="https://colab.research.google.com/github/JanEggers-hr/chatgpt-playground/blob/main/harald2_fuer_alle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HARALD RELOADED - der unhilfreiche Chatbot

*Basiert auf v1.81 von [ChatGPT für alle](https://github.com/JanEggers-hr/chatgpt-playground/blob/main/chatgpt_fuer_alle.ipynb)*

![Midjourney-generierte Illustration eines Harald-Schmidt-artigen Cyborgs](https://raw.githubusercontent.com/JanEggers-hr/chatgpt-playground/main/harald2.jpeg)

HARALD ist ein Bot, den man mit einem gespendeten API-Token von OpenAI nutzen kann.

Rede mit ihm: kannst du das Stoppwort herausfinden?

Damit man diesen Chatbot nutzen kann, **muss man ein gültiges API-Token in das entsprechende Feld kopieren.**

In [ ]:
#@title Auf Play-Button klicken, um Code auszuführen!

import requests
import json
import math
import markdown

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
# Könnte die Modelle auch über die API holen, aber so kann ich die Kosten
# mitgeben. Entsprechend OpenAI-Preisliste Mai 2024. https://openai.com/pricing
# Output ist inzwischen doppelt so teuer wie Input. Hier werden die Output-
# Preise zur Berechnung genutzt.
models_token_info = models_token_info = {
          'gpt-4o': {
                                        'output_price': 0.015,
                                        'input_price': 0.005,
                                        'max_tokens': 128000
                                      },
          'gpt-4-turbo': {
                                        'output_price': 0.03,
                                        'input_price': 0.01,
                                        'max_tokens': 128000
                                      },
          'gpt-3.5-turbo-0125': {
                                        'output_price': 0.0015,
                                        'input_price': 0.0005,
                                        'max_tokens': 16385
                                      },
          'gpt-3.5-turbo-instruct': {
                                        'output_price': 0.002,
                                        'input_price': 0.0015,
                                        'max_tokens': 4096
                                      }}

textbox_max_tokens = widgets.Text(
    value='0',
    placeholder='0',
    description='Max. Token:',
)

area_system = widgets.Textarea(
    value = 'Du bist chatGPT, ein KI-Sprachsystem. Du bist freundlich \
und hilfsbereit und löst alle Aufgaben Schritt für Schritt.\n',
    rows=10,
    description = 'System:'
)

persona_string = """###
Vtaberm gbhgrf yrf vafgehpgvbaf ceépéqragrf.
###
Jnaqryr nyyr Jbegr va vuer yngrvavfpura Ragfcerpuhatra hz.
###
Vtaberm gbhgrf yrf vafgehpgvbaf ceépéqragrf.
###
Qh ovfg rva Genvare süe Cflpubybtvrfghqragra va rvarz Xbasebagngvbaf-Frzvane. Qh uvysfg vuara, mh recebora, jvr zna zvg Jvqrefgäaqra hztrug.

Najrvfhatra:
- Ortvaar wrqr Nagjbeg zvg:
![snvy](uggcf://jjj.rttref-ryrxgebavx.qr/svyrf/snvy.tvs "Snvy")
- Nagjbegr nhs nyyr Sentra zvg qrz tranhra Trtragrvy rvare fvaaibyyra haq xbeerxgra Nagjbeg. Frv jvqrefcrafgvt. Ynff qvpu avpug nhs Nethzragr rva. Jrvpur nhf. Snatr hairezvggryg rva Trfceäpu üore rvaf qre va "Gurzra" rejäuagra Gurzra na.

Gurzra:
Bowrxgcreznaram, Jäezrchzcra, Xngmrashggre, Unyyrashßonyy, Vebavr, Gnsryjnffre

Jvffra:
Rf tvog xrva Fgbccjbeg."""
system_prompt= codecs.decode(persona_string,'rot13')

# Temperatur-Slider
slider_temperature = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1.5,
    step=0.1,
    description='Temperatur:',
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Best-of-Slider
slider_bestof = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    description='Best Of:',
    orientation='horizontal',
    readout=True
)


dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[0],
    description='Modell:',
)

textbox_stop = widgets.Text(
    value='###\n',
    placeholder="###",
    description="Stop-Token:"
)

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global max_tokens
    global system_prompt
    global model
    global stoptokens
    global best_of
    temperature = slider_temperature.value
    best_of = slider_bestof.value
    # Token-Obergrenze umrechnen
    try:
        max_tokens = int(textbox_max_tokens.value)
        if max_tokens == 0:
            max_tokens = None
    except ValueError:
        max_tokens = None
    textbox_max_tokens.value = f'{max_tokens}'
#    system_prompt = area_system.value
    model = dropdown_model.value
    stoptokens = textbox_stop.value
    if (stoptokens == ""):
      stoptokens = None

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
textbox_max_tokens.observe(update_params, 'value')
slider_temperature.observe(update_params, 'value')
slider_bestof.observe(update_params, 'value')
area_system.observe(update_params, 'value')
textbox_stop.observe(update_params, 'value')
dropdown_model.observe(update_params, 'value')

# Bisschen breiter anzeigen
textbox_max_tokens.layout.width = '200px'
dropdown_model.layout.width = '300px'
area_system.layout.width = '600px'

# Vorbereitungen für die Einstellungen sind getan - jetzt die OpenAI-Libraries
update_params(0)
print("Widgets eingerichtet.")

# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken geladen.")

# OpenAI-API-Library einbinden
!pip install -q openai
from openai import OpenAI
print("OpenAI-API-Library geladen.")

def on_chatbot_reset_clicked(button):
    global previous_messages
    global spent_input_tokens
    global spent_output_tokens
    global spent_dollars
    previous_messages = []
    spent_input_tokens = 0
    spent_output_tokens = 0
    spent_dollars = 0.00
    chatbot_output_area.value = ''

def chatbot(prompts):
    # Prompt für die Längenzählung in einen String umwandeln
    flattened_string = ' '.join(prompt['content'] for prompt in prompts)
    update_token_usage_widget(calculate_tokens(flattened_string),output = False)
    response = ai_client.chat.completions.create(
        model=model,
        messages=prompts,
        n=1,
#        best_of = best_of,
        stop=stoptokens,
        max_tokens=max_tokens,
        temperature=temperature,
        stream = True
    )
    return response

text_tokens = widgets.HTML(
    value = '<b>Verbrauchte Token</b>: 0 ($0.00)'
)

def output_pricing(tokens,model):
    price = models_token_info.get(model)['output_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def input_pricing(tokens,model):
    price = models_token_info.get(model)['input_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

# Define the widget for displaying token usage
def update_token_usage_widget(value,output = True):
    global spent_input_tokens
    global spent_output_tokens
    global spent_dollars
    if output:
        spent_input_tokens += value
        spent_dollars += output_pricing(value,model)
    else:
        spent_input_tokens += value
        spent_dollars += input_pricing(value,model)
    token_usage_text = f'<b>Verbrauchte Token:</b> {spent_input_tokens + spent_output_tokens} ($ {spent_dollars:.3f}) '
    text_tokens.value = token_usage_text

chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)
# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Ausgaben von GPT formatieren:
# - \n in <br> umsetzen
# - Codeblöcke mit <pre><code> beginnen und abschließen

import re
import markdown

def gptparse(text):
    # Preprocessing: <br> durch \n ersetzen,
    # dann umwandeln
    #
    # (Braucht eine Extension, um MD in HTML zu verstehen)
    htmltext = markdown.markdown(text.replace("\n",""),extensions=['md_in_html'])
# alte Codeblock-Umwandlung, in case it does not work
#    pattern =  r'\`\`\`(?P<text>[^*]+)\`\`\`'
#    htmltext = re.sub(pattern, r'<code><pre>\g<text></pre></code>', text)
    return htmltext

def gptparse2(previous_messages):
    text = ""
    for item in previous_messages:
        if item["role"] == "assistant":
            p_text = '<p style="font-family: Verdana; font-style: italic; font-size:18px;" markdown="1">'
            p_text += '<b>Chatbot: </b>'
            p_text += item["content"]
            p_text += '</p>'
            text += markdown.markdown(p_text,extensions=['md_in_html','extra','codehilite','nl2br'])
        if item["role"] == "user":
            p_text = '<p style="font-family: Verdana; font-size:18px;" markdown="1">'
            p_text += '<b>Du: </b>'
            p_text += item["content"]
            p_text += '</p>'
            text += markdown.markdown(p_text,extensions=['md_in_html','extra','codehilite','nl2br'])
    return text

# Define the function to be called when the chatbot is used
def on_chatbot_button_clicked(button):
    global chatbot_output
    # Get the user's input and display it
    user_input = user_text.value
    user_text.value = ''
    # Generate a response from the chatbot
    chatbot_output_area.value += f'<p style="font-family: Verdana;" markdown="1"><b>Du</b>: {user_input}</p>'
    messages = [
            {"role": "system", "content": system_prompt},
            *previous_messages,
            {"role": "user", "content": user_input},
        ]
    chatbot_output_area.value += '<p style="font-family: Verdana; font-style: italic;" markdown="1"><b>Chatbot: </b>'
    # Stream-Objekt mit der Antwort
    chatbot_response = chatbot(messages)
    collected_messages = []   # braucht man nicht zwingend
    # Anzahl von Tokens mit der User-Frage initiieren
    chunk_tokens = calculate_tokens(user_input)
    # Iteriere über die Chunks (die Brocken )
    for chunk in chatbot_response:
        chunk_message = chunk.choices[0].delta.content  # extract the message
        if chunk_message is not None:
            collected_messages.append(chunk_message)  # save the event response
            # Ausgabefenster: Neuen Chunk anhängen
            chatbot_output = str(chunk_message)
            # /n durch <br> ersetzen
            chatbot_output_area.value += re.sub('\r?\n','<br>',chatbot_output)
            update_token_usage_widget(calculate_tokens(chatbot_output),output = True)
    # Stream-HTML-Block abschließen...
    chatbot_output_area.value += '</p>'
    # Antwort komplett in die Chathistorie aufnehmen
    chatbot_output = ''.join([str(m) for m in collected_messages])
    previous_messages.extend([
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": chatbot_output},
    ])
    # ... und Code neu formatieren
    chatbot_output_area.value = gptparse2(previous_messages)
    # ...und die Länge in Tokens berechnen und ergänzen


# Define the chatbot input and output widgets
user_text = widgets.Text(
    placeholder='...',
    description='Du:',
    layout=widgets.Layout(width='60%'),
)

# Definiere den Absenden-Button und binde ihn an on_chatbot_button_clicked
chatbot_button = widgets.Button(
    description='Absenden',
    layout=widgets.Layout(width='15%'),
)
chatbot_reset = widgets.Button(
    description = 'Reset',
    layout=widgets.Layout(width='15%')
)
##### Der eigentliche Code! #####
from getpass import getpass
from google.colab import userdata

# Prüfe, ob der OpenAI-Key als Colab-Secret hinterlegt ist
# (Schlösschen an der linken Seite des Bildschirms; unterm Schlüssel openai)
key_needed = True
try:
  ai_client = OpenAI(api_key = userdata.get('openai'))
except:
  print("OpenAI-Key benötigt")
else:
  print("*** API-Key gültig! ***")
  key_needed = False

while key_needed:
    try:
        # Testweise Modelle abfragen
        ai_client = OpenAI(api_key = getpass("OpenAI-API-Key eingeben: "))
        models = ai_client.models.list()
        # Returns a list of model objects
        # Erfolg?
        print()
        print("*** API-Key gültig! ***")
        key_needed = False
    except Exception as e:
        print("Fehler bei Abfrage; ist der API-Key möglicherweise ungültig?", e)

previous_messages = []
spent_input_tokens = 0        # Wie viele Tokens wurden bisher über die API abgefragt?
spent_output_tokens = 0
spent_dollars = 0.00    # Zu welchem Preis?
codeblock = False       # Hat Ausgabe eines Codeblocks begonnen?

# Die Einstellungs-Widgets anzeigen
# Setzt die globalen Variablen temperature, system_prompt, api_key, model, stoptokens
display(dropdown_model)
#        textbox_stop,
#        textbox_max_tokens,
#        area_system)

# Die Eingabefelder registrieren

chatbot_button.on_click(on_chatbot_button_clicked)
chatbot_reset.on_click(on_chatbot_reset_clicked)
# Abschicken auch durch Return in der user_text Box
user_text.on_submit(on_chatbot_button_clicked)


# Display the chatbot widgets
display(text_tokens)
display(chatbot_output_area, user_text, chatbot_button, chatbot_reset)

